In [1]:
import tensorflow as tf 
from tensorflow.keras.models import Model 
import tensorflow.keras.layers as L

In [2]:
def conv_block(x, num_filters): 
    x = L.Conv2D(num_filters, 3, padding="same")(x) 
    x = L.BatchNormalization()(x) 
    x = L.Activation("relu")(x)

    x = L.Conv2D(num_filters, 3, padding="same")(x) 
    x = L.BatchNormalization()(x) 
    x = L.Activation("relu")(x)

    return x

In [3]:
def encoder_block(x, num_filters): 
    x = conv_block(x, num_filters)
    p = L.MaxPooling2D((2, 2))(x)
    return x, p


In [4]:
def attention_gate(g, s, num_filters): 
    Wg = L.Conv2D(num_filters, 1, padding="same")(g) 
    Wg = L.BatchNormalization()(Wg) 

    Ws = L.Conv2D(num_filters, 1, padding="same")(s)
    Ws = L.BatchNormalization()(Ws)

    output = L.Activation("relu")(Wg + Ws) 
    output = L.Conv2D(num_filters, 1, padding="same")(output) 
    output = L.Activation("sigmoid")(output) 

    return output * s

In [7]:
def decoder_block(x, s, num_filters): 
    x = L.UpSampling2D(interpolation="bilinear")(x) 
    s = attention_gate(x, s, num_filters)
    x = L.Concatenate()([x, s])
    x = conv_block(x, num_filters) 
    return x

In [11]:
def Attention_Unet(input_shape, num_classes): 
    inputs = L.Input(input_shape) 

    """Encoder"""
    s1, p1 = encoder_block(inputs, 64) 
    s2, p2 = encoder_block(p1, 128) 
    s3, p3 = encoder_block(p2, 256) 

    b1 = conv_block(p3, 512) 
    
    """Decoder"""
    d1 = decoder_block(b1, s3, 256) 
    d2 = decoder_block(d1, s2, 128)
    d3 = decoder_block(d2, s1, 64)

    """Output"""
    outputs = L.Conv2D(num_classes, 1, padding="same", activation="softmax")(d3)

    """Model"""
    model = Model(inputs, outputs, name="Attention-UNet")
    return model 

In [14]:
input_shape = (600, 600, 512) 
model = Attention_Unet(input_shape, 11)
model.summary()

Model: "Attention-UNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 600, 600,  │       0 │ -                    │
│ (InputLayer)        │ 512)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_64 (Conv2D)  │ (None, 600, 600,  │ 294,976 │ input_layer_4[0][0]  │
│                     │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 600, 600,  │     256 │ conv2d_64[0][0]      │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_56       │ (None, 600, 600,  │       0 │ batch_normalization… │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_65 (Conv2D)  │ (None, 600, 600,  │  36,928 │ activation_56[0][0]  │
│                     │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 600, 600,  │     256 │ conv2d_65[0][0]      │
│ (BatchNormalizatio… │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_57       │ (None, 600, 600,  │       0 │ batch_normalization… │
│ (Activation)        │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pooling2d_12    │ (None, 300, 300,  │       0 │ activation_57[0][0]  │
│ (MaxPooling2D)      │ 64)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_66 (Conv2D)  │ (None, 300, 300,  │  73,856 │ max_pooling2d_12[0]… │
│                     │ 128)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 300, 300,  │     512 │ conv2d_66[0][0]      │
│ (BatchNormalizatio… │ 128)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_58       │ (None, 300, 300,  │       0 │ batch_normalization… │
│ (Activation)        │ 128)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_67 (Conv2D)  │ (None, 300, 300,  │ 147,584 │ activation_58[0][0]  │
│                     │ 128)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 300, 300,  │     512 │ conv2d_67[0][0]      │
│ (BatchNormalizatio… │ 128)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ activation_59       │ (None, 300, 300,  │       0 │ batch_normalization… │
│ (Activation)        │ 128)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pooling2d_13    │ (None, 150, 150,  │       0 │ activation_59[0][0]  │
│ (MaxPooling2D)      │ 128)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_68 (Conv2D)  │ (None, 150, 150,  │ 295,168 │ max_pooling2d_13[0]… │
│                     │ 256)              │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ batch_normalizatio… │ (None, 150, 150,  │   1,024 │ conv2d_68[0][0]    

 Total params: 8,437,003 (32.18 MB)

 Trainable params: 8,429,579 (32.16 MB)

 Non-trainable params: 7,424 (29.00 KB)